In [1]:
import pandas as pd
import numpy as np

In [10]:
df= pd.read_csv('상수합정서교1_공공데이터합치기_완성.csv', encoding='utf8')

In [11]:
df.tail()

,name,cate_1,cate_2,cate_3,dong,lon,lat,cate_mix,naver_keyword,naver_map_url,naver_store_type,naver_star_point,naver_star_point_qty,naver_blog_review_txt,naver_blog_review_qty
830,북까페산책,커피점/카페,커피전문점/카페/다방,비알콜 음료점업,서교동,37.555201,126.919633,커피점 카페커피전문점 카페 다방비알콜 음료점업,서교동%20북까페산책,https://m.place.naver.com/restaurant/37996760,북카페,4.62,102,룰루랄라 홍대 북카페에 다녀왔어영 사실 IVP는 대학생 때 부터 종종 갔었던 곳이에...,41.0
831,돌담커피,커피점/카페,커피전문점/카페/다방,비알콜 음료점업,서교동,37.554735,126.926793,커피점 카페커피전문점 카페 다방비알콜 음료점업,서교동%20돌담커피,https://m.place.naver.com/restaurant/1834600905,"카페,디저트",4.52,27,#홍대카페 #돌담커피 #돌담coffee #홍대카페추천 #홍대테이크아웃커피 #홍대ta...,3.0
832,섬마을,일식/수산물,음식점-일식,일식 음식점업,서교동,37.549823,126.920763,일식 수산물음식점-일식일식 음식점업,서교동%20섬마을,https://m.place.naver.com/restaurant/438356658,"초밥,롤",4.38,112,컨설팅 저녁식사로 첨 방문했다가 넘 맛있어서 일주일만도 안되어 재방문한 홍대 섬마을...,71.0
833,나팔,패스트푸드,피자전문,"피자, 햄버거, 샌드위치 및 유사 음식점업",서교동,37.549810,126.920311,"패스트푸드피자전문피자, 햄버거, 샌드위치 및 유사 음식점업",서교동%20나팔,https://m.place.naver.com/restaurant/1501211213,피자,4.90,104,"시동생이랑 김뮤키랑 합정고깃집에서 배 터지게 먹고 2차를 가자, 고 해서 홍대 쪽으...",115.0
834,지코바양념치킨,유흥주점,호프/맥주,기타 주점업,서교동,37.552904,126.920685,유흥주점호프 맥주기타 주점업,서교동%20지코바양념치킨,https://m.place.naver.com/restaurant/1332461169,"치킨,닭강정",4.46,321,홍대 배달 맛집 지코바 순살 양념 치킨 얼마전 생일이어서 기프티콘 선물을 많이 받았...,50.0


In [12]:
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.metrics.pairwise import cosine_similarity

count_vect_category = CountVectorizer(min_df=1, ngram_range=(1,2))
place_category = count_vect_category.fit_transform(df['cate_mix']) 
place_simi_cate = cosine_similarity(place_category, place_category) 
place_simi_cate_sorted_ind = place_simi_cate.argsort()[:, ::-1]

In [13]:

count_vect_review = CountVectorizer(min_df=2, ngram_range=(1,2))
place_review = count_vect_review.fit_transform(df['naver_blog_review_txt']) 

place_simi_review = cosine_similarity(place_review, place_review)
place_simi_review_sorted_ind = place_simi_review.argsort()[:, ::-1]

In [14]:
print('코사인유사도결과:',place_simi_review.shape)

코사인유사도결과: (835, 835)


In [15]:


place_simi_co = (
                 + place_simi_cate * 0.3 
                 + place_simi_review * 0.1 
                 )



place_simi_co_sorted_ind = place_simi_co.argsort()[:, ::-1] 



def find_simi_place(df, sorted_ind, place_name, top_n=5):
    
    place_title = df[df['name'] == place_name]
    place_index = place_title.index.values
    similar_indexes = sorted_ind[place_index, :(top_n)]
    similar_indexes = similar_indexes.reshape(-1)
    return df.iloc[similar_indexes]

find_simi_place(df, place_simi_co_sorted_ind, '당인리책발전소', 6)

,name,cate_1,cate_2,cate_3,dong,lon,lat,cate_mix,naver_keyword,naver_map_url,naver_store_type,naver_star_point,naver_star_point_qty,naver_blog_review_txt,naver_blog_review_qty
545,당인리책발전소,커피점/카페,커피전문점/카페/다방,비알콜 음료점업,서교동,37.555682,126.911126,커피점 카페커피전문점 카페 다방비알콜 음료점업,서교동%20당인리책발전소,https://m.place.naver.com/restaurant/1464359219,북카페,4.54,851,"당인리 책발전소를 다녀왔다. 책 좋아하고, 커피 좋아하고. 그래서 가보고 싶었던 당...",634.0
463,더블하모니,커피점/카페,커피전문점/카페/다방,비알콜 음료점업,서교동,37.554221,126.913921,커피점 카페커피전문점 카페 다방비알콜 음료점업,서교동%20더블하모니,https://m.place.naver.com/restaurant/35769418,카페,4.31,160,보통 망원동에 핫플은 망원역 1번출구 쪽이나 망원역 2번출구 주변이라서 망원역 과 ...,159.0
654,고래상점,커피점/카페,커피전문점/카페/다방,비알콜 음료점업,서교동,37.548583,126.919140,커피점 카페커피전문점 카페 다방비알콜 음료점업,서교동%20고래상점,https://m.place.naver.com/restaurant/38300449,카페,4.36,212,오빠랑 힙정 나들이~! 방탈출 하고 시간이 좀 애매하게 남아서 그냥 보이는 카페 들...,268.0
429,더클로짓,커피점/카페,커피전문점/카페/다방,비알콜 음료점업,합정동,37.544516,126.914819,커피점 카페커피전문점 카페 다방비알콜 음료점업,합정동%20더클로짓,https://m.place.naver.com/restaurant/1074319360,"카페,디저트",4.59,31,The closet 매일 영업 월 17 : 00 - 22 : 00 화-일 13 : ...,44.0
720,유들렌,커피점/카페,커피전문점/카페/다방,비알콜 음료점업,서교동,37.554558,126.926116,커피점 카페커피전문점 카페 다방비알콜 음료점업,서교동%20유들렌,https://m.place.naver.com/restaurant/1201184386,"카페,디저트",4.66,244,얼마전에 홍대근처 지나가다가 귀여운 카페 발견해서 다녀왔어요:) 입구 쪽에 링거 꽂...,148.0
558,컨택트,커피점/카페,커피전문점/카페/다방,비알콜 음료점업,서교동,37.550274,126.918821,커피점 카페커피전문점 카페 다방비알콜 음료점업,서교동%20컨택트,https://m.place.naver.com/restaurant/1825202007,"카페,디저트",4.50,316,점심으로 우동 호로록 하고!! 일보고나서 이대로 가긴 아쉬워 커피타임을 가지기로 한...,161.0
